In [2]:
%load_ext autoreload
%autoreload 2

import lzma
import dill
import json
import pandas as pd
import shap
from tqdm.auto import tqdm
import numpy as np
import tensorflow as tf
from pathlib import Path

import scienceplots

import matplotlib.pyplot as plt
import matplotlib as mpl

from matal import load_model, load_data, auto_log, DATA_DIR, MODEL_DIR, augmente_data
from matal.ml import CompositionGenerator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
X_COLS = ['MMT', 'CNF', 'GEL', 'GLY']
Y_COLS = ['VLTransmittance', 'UVTransmittance', 'IRTransmittance',
          'AshAreaRatio', 
          'Stress', 'Strain', 'Modulus']

round_df = load_data('14.train_data__da0.table.csv')
db_model = load_model('svm')

def feas_func(comps):
    return np.array([db_model.predict([row[0:3]])[0] > 0.75 for row in comps])

def var_func(comps):
    return np.sum((model.predict_variance(comps) / model.predict(comps)) for _, model in vmodel.items())

vmodel = load_model('vmodels_final')
vmodel

Trying to unpickle estimator SVR from version 0.24.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations


{'VLTransmittance': MOVotingRegressor(estimators=[('14_VLTransmittance_1_VJ3HSBM3',
                                MLPNetwork(input_norm=True,
                                           name='14_VLTransmittance_1_VJ3HSBM3',
                                           output_scale=100)),
                               ('14_VLTransmittance_2_PCSTZBJN',
                                MLPNetwork(input_norm=True,
                                           name='14_VLTransmittance_2_PCSTZBJN',
                                           output_scale=100)),
                               ('14_VLTransmittance_3_VUUFLQTS',
                                MLPNetwork(input_norm=True,
                                           name='14_VLTransmittance_3_VUUFLQTS',
                                           output_scale=100)),
                               ('14_VLTransmittance_4_VJ3HSBM3',
                                MLPNetwork(input_norm=True,
                                           name='

In [4]:
# Generate composition for the next round
g = CompositionGenerator(materials=X_COLS, n_comps=10, n_iters=10000, random_state=1, 
                         existing_comps=round_df[X_COLS],
                         perf_func=var_func,
                         feas_func=feas_func,
                        )
comps = g.optimize()
comps

  0%|          | 0/10000 [00:00<?, ?it/s]

,MMT,CNF,GEL,GLY
0,0.538110,0.018839,0.011944,0.431108
1,0.249156,0.313961,0.159554,0.277329
2,0.140986,0.405609,0.404262,0.049143
3,0.439440,0.184277,0.024848,0.351435
4,0.376453,0.362203,0.135165,0.126178
5,0.072733,0.462376,0.330480,0.134411
6,0.300970,0.314698,0.205421,0.178911
7,0.465074,0.018343,0.119339,0.397243
8,0.491731,0.086936,0.032676,0.388658
9,0.012244,0.423587,0.413456,0.150713
